# TF的MNIST入门

本文适合对机器学习和TF都不太了解的新手。

如果读者已经了解MNIST和Softmax回归的相关知识，可以直接跳往[TF的MNIST提高](./TF的MNIST提高.ipynb)。

很多人生平第一次编程，就是打印“hello world”，MNIST好比机器学习的Hello-World。

MNIST是一种入门级的计算机视觉数据集，包含各种手写数字图片。

![](./图表/MNIST.png "手写数字图片示例")

每一张图包含一个数字以及对应的标签，该标签保存真实的数值。上面四张图的标签分别是“5、0、4、1”。

要讲述的就是训练一个机器学习模型来预测图片里的数字。

目的不是设计一个世界一流的复杂模型，尽管之后给出的代码可以实现一流的预测模型，而是要介绍如何使用TF来训练模型。

从一个很简单的数学模型起步，叫做Softmax回归。

对应的实现代码很短，且真正有意思的内容只有三行。但是，理解这些代码的设计思想非常重要：TF工作流程和机器学习的基本概念。本文会很详细地介绍这些代码的实现原理。

## MNIST数据集

来自[Yann LeCun](http://yann.lecun.com/exdb/mnist/)，这里提供代码用于自动下载和配置这个数据集。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


下载的数据被分为两部分：

- 六万行的训练数据集（mnist.train）
- 一万行的测试数据集（mnist.test）

这样切分很重要，在机器学习模型设计时必须有一个单独的测试数据集、不用于训练而用于评估整个模型的性能，从而更容易把设计的模型推广到别的数据集上（泛化）。

正如前面条的一样，每个MNIST数据单元由两部分组成：

- 包含一个手写数字的图片
- 图片对应的标签标识真值

将这些图片设为 $xs$ ，把这些标签设为 $ys$ 。

训练数据集和测试数据集都包含 $xs$ 和 $ys$ ，比如训练数据集的图片是mnist.train.images，训练数据集的标签是mnist.train.labels。

**每张图片包含28*28个像素点**。

可以用像素矩阵来表示一张图片。

![](./图表/MNIST-Matrix.png "数字图片的矩阵表示")

把这个矩阵展开成一个向量，长度是784（28*28）。

如何展开这个矩阵（行列甚至元素的顺利）不重要，只要保证各个图片采用相同的方式展开。

站在这个角度考虑，MNIST数据集的图片就是784维向量空间中的点集，并且拥有比较[复杂的结构](http://colah.github.io/posts/2014-10-Visualizing-MNIST/ "目测MNIST：降维测试")（此类数据的可视化是计算密集型的）。

展开矩阵会丢失维度结构信息，这显然不理想，最优秀的计算机视觉方法会挖掘并利用这些结构信息，后续会介绍，本文暂时忽略，只介绍简单数学模型，Softmax回归不会利用这些结构信息。

因此，在MNIST训练数据集中，mnist.train.images是一个形状为 $[60000, 784]$ 的张量，第一个维度用来索引图片，第二个维度用来索引某张图片中的像素点。在此张量中，每个元素都表示某张图片里某个像素的强度值，介于 $[0, 1]$ 之间。

![](./图表/mnist-train-xs.png)

相应的MNIST数据集的标签式介于 $[0, 9]$ 的数字，用来描述给定图片里表示的数字。

为了用于本文，标签数据是“one-hot”向量，即某一位数字是 $1$ 其余位都是 $0$ 。

所以在本文中，数字 $n$ 将表示成一个只在第n维（从0起）为 $1$ 的十维向量。

例如，标签 $0$ 表示为 $[1,0,0,0,0,0,0,0,0,0]$ 。

因此，mnist.train.labels是一个 $[60000, 10]$ 的矩阵。

![](./图表/mnist-train-ys.png)

接下来，着手构建模型！

## Softmax回归

都知道MNIST的每一张图片都表示一个 $[0, 9]$ 的自然数字。

希望得到给定图片每个数字的概率，如模型推测真值是9的图片包含9的概率是80%、包含8的概率是5%（因为8和9上半部分的圈近似），包含别的数字的概率更小。

这是应用Softmax回归的经典案例。

**Softmax可以用来给不同的对象分配概率，即便以后训练更加精细的模型时，最后一步也需要用Softmax来分配概率**。

- 第一步

    为了得到给定图片属于某个特定数字的证据，对图片像素值加权求和，若其中某个像素具有很强的证据表明该图片不属于这个特定数字，相应的权值为负数，想法是正数。
    
    下图展示某模型学习到的图片上每个像素对特定数字的权值，红色为负数、蓝色为正数。
    
    ![](./图表/softmax-weights.png)
    
    还需要添加一个额外的偏置，因为输入往往会带有一些无关的干扰量，因此对于给定的图片 $x$ 代表数字 $i$ 的证据表示为： $$evidence_i = \sum_jW_{i,j}x_j + b_i$$
    
    其中， $W_i$ 代表权重， $b_i$ 代表数字 $i$ 的偏置， $j$ 代表给定图片 $x$ 的像素索引（用于像素加权求和）。
    
    接着用Softmax把这些证据转换成概率 $y$ ： $$y=softmax(evidence)$$
    
    这里的 $softmax$ 可以看成一个激励函数（或链接函数），把定义的线性函数的输出转换成想要的格式，也就是关于十个数字的概率分布。这样，给定一张图，它对于每个数字的吻合度可以被Softmax转换成一个概率值。
    
    Softmax的定义为： $$softmax(x)=normalize(\exp(x))$$
    
    展开等式右侧的子式，可以得到： $$softmax(x)_i=\frac{\exp(x_i)}{\sum_j\exp(x_j)}$$
    
    更多时候把Softmax模型函数定义为前一种形式：把输入值当成幂指数求值，接着正则化这些结果值。
    
    这个幂运算表明：更大的证据对应更大的假设模型里面的乘数权重值；反之，拥有更少的证据意味着在假设模型里拥有更小的乘数系数。
    
    假设模型里的权值不可以是零或负数。
    
    Softmax然后会正则化这些权重值，使得总和等于一，以此构造有效地概率分布。
    
    *更多有关Softmax函数的解释，[参考Michael Nieslen的书](http://neuralnetworksanddeeplearning.com/)，其中有Softmax的交互式可视化阐述*。

### Softmax归回图解

- 将输入 $xs$ 加权求和
- 分别加入偏置
- 再输入到softmax函数中

![](./图表/softmax-regression-scalargraph.png)

写成等式：

![](./图表/softmax-regression-scalarequation.png)

运用线性代数的方法改写：

![](./图表/softmax-regression-vectorequation.png)

更紧凑的写法： $$y=softmax(W_x + b)$$

### 实现Softmax回归模型

为了让Python实现高效的数值计算，通常调用函数库，如NumPy等，会把类似矩阵乘法这样的复杂运算交给外部语言实现。

不幸的是，从外部计算切换回Python的每一个操作，仍然是一个很大的开销。

如果用GPU来进行外部计算，**这种**开销会更大。

用分布式的计算方式，会开销更多资源来传输数据。

重点来了：TF的确把复杂运算放在Python之外完成，为了避免前述的开销，做了进一步完善。TF不单独地执行单一的复杂运算，而是让设计者先用图描述一系列可交互的计算操作，然后全部一起在Python之外运行！*类似的运行方式，可见于不少的机器学习库*。

首先导入TF：

    import tensorflow as tf

然后通过操作符号变量来描述这些可交互的操作单元，例如：

    x = tf.placeholder(tf.float32, [None, 784])

此时的x不是一个特定值，而是一个占位符，在TF运行计算时输入这个值，这里希望能够输入任意数量的MNIST图像，每一张展开成784维的向量，用2维浮点数张量来表示这些图，这个张量的形状是[None, 784]（这儿的None表示第一个维度可以是任意（正整数）的）。

模型需要权重值和偏置量，当然可以将两者当作另外的输入（使用占位符），但TF有个更好的方法来表示：Variable。

一个Variable代表一个可修改的张量，存在于TF用于描述可交互性操作的图中。可以用于计算输入值，也可以在计算中被修改。对于各种机器学习应用，一般都有模型参数，可以用Variable表示。

    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10])

赋予tf.Variable不同的初始值来创建不同的Variable：这里都是用全零的张量来初始化W和b，因为训练模型就是要学习W和b的值，初值可随意设置。

注意：

- W的维度是[784, 10]，因为想要用784维的图片向量乘以W得到一个10维德证据值向量，每一位对应不同的数字。
- b的形状是[10]，因此可以直接将它加到输出上面。

现在，仅需一行代码即可实现Softmax模型：

    y = tf.nn.softmaxt(tf.matmul(x, W) + b)

- 用tf.matmul(x, W)表示x乘以W，对应前述等式的$W_x$，这里的x是一个2维张量，有多路输入。
- 再加上b。
- 把和输入到tf.nn.softmax函数里。

至此，用几行简短的代码来设置变量在前、一行代码来定义模型在后。

TF不仅能让Softmax回归模型计算变得特别简单，它也用这种非常灵活的方式来描述别的各种数值计算，从机器学习模型到物理学模型模拟仿真。模型一旦被定义好，就可以在不同的设备上运行：CPU/GPU，甚至手机！

### 训练模型

凡是闭合系统，有产出就必须有消费，有发送就必须有接收，有训练就不必须有评价。

机器学习中必须评价一个模型训练的效果的优劣。通常定义指标来表示模型的不良程度，这个指标称为成本（cost）或者损失（loss），然后尽量最小化这个指标。

一种的、漂亮的成本函数是“交叉熵（Cross-Entropy）”，产生于信息论中的信息压缩编码技术，后来演变成为博弈论到机器学习等相关领域的重要技术手段。定义如下： $$H_{y^{'}}(y)=-\sum_{i}y_i^{'}\log(y_i)$$

其中， $y$ 是预测的概率分布， $y^{'}$ 是实际的概率分布（输入的One-Hot Vector）。

比较粗糙的理解是，交叉熵是用来衡量“预测用于描述真相的低效性”。更详细的**交叉熵理论**可[参考这里](http://colah.github.io/posts/2015-09-Visual-Information/)。

为了计算交叉熵，要添加一个新的占位符用于输入正确值：

    y_ = tf.placeholder("float", [None, 10])

接着可以用 $-\sum y^{'}\log(y)$ 计算交叉熵：

    ce = - tf.reduce_sum(y_ * tf.log(y))

先用tf.log计算y的每个元素的对数；接着把y_的每一个元素和tf.log(y)的对应元素相乘；最后用tf.reduce_sum计算张量的所有元素的总和。

注意：这里的交叉熵不仅仅用来衡量单一的一对预测值和真实值，而是所有100幅图片的交叉熵的总和。对于100个数据点的预测表现比单一的数据点的表现，能更好地描述模型的性能。

现在知道需要模型做些什么，用TF训练它非常容易。因为TF拥有一张描述各个计算单元的图，可以自动调用[反向传播（backpropagation）](http://colah.github.io/posts/2015-08-Backprop/)算法来有效地确定变量如何影响要最小化的那个成本（损失）的。然后TF会以设计者选择的优化算法迭代变量以降低成本（损失）。

    trains = tf.train.GradientDescentOptimizer(0.01).minimize(ce)

这里要求TF用梯度下降算法以0.01的学习率最小化交叉熵。

梯度下降算法是一个简单的学习过程，TF只需要将每个变量一点点地往*使成本（损失）不断变小的方向*移动。

当然，TF提供了许多[优化算法](./TF的Train.ipynb)，只需要简单地调整一行代码就可以采用别的算法。

在此TF实际上所做的是，它会在后台给设计者描述计算的那幅张量图里添加一系列新的计算操作单元用于实现反向传播算法和梯度下降算法，然后返回的只是一个单一的操作，当运行这个操作时，它用梯度下降算法训练模型、微调变量，不断减少成本（损失)。

设置好模型，在运算之前，需要添加一个初始化所创建的变量的操作。

    init = tf.initialize_all_variables() # deprecated, use global_variables_initializer instead

现在可以在一个Session里启动模型并初始化变量。

    sess = tf.Session()
    sess.run(init)

然后进行模型训练，迭代1000次！

    for i in range(1000):
      batch_xs, batch_ys = mnist.train.next_batch(100)
      sess.run(trains, feed_dict={x: batch_xs, y: batch_ys})

该循环的每个步骤中，都会随机抓取训练数据集合的100个批处理数据点，然后用这些数据点作为参数替换之前的占位符来运行trains。

使用一小部分随机数据点来进行训练模型称为“随机（猜测）训练（stochastic training）”，这里更确切的说是“随机梯度下降训练”。

理想情况下，希望用所有数据进行每一步的训练，因为这能产出更好的训练结果，但显然这需要很大的计算开销，所以，每一次训练使用不同的数据子集，这样做既可以减少计算开销，又可以最大化地学习到数据集的总体特征。

### 评估模型

接下来讲述如何评估模型的性能。

首先找出那些预测正确的标签。

tf.argmax是一个非常有用的函数，它能给出某个张量对象在某一维度上数据最大最所在的索引。

由于标签向量由0和1组成，因此最大值1所在的索引位置就是类别标签。比如，tf.argmax(y, 1)返回的是模型对于任一输入x预测到的标签值，而tf.argmax(y_, 1)代表正确的标签。可以用tf.equal来检测预测和真实标签匹配否（索引一致表示匹配）。

    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

这行代码给出一组布尔值，为了确定正确率，将布尔值转换为浮点数，然后取平均值。例如，[True, False, True, True]变成[1.0, 0.0, 1.0, 1.0]，取平均值后得到0.75。

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float))

最后，计算所学习到的模型在测试数据集合上的正确率。

    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

结果最终大约是0.91。

这个结果好吗？是的，并不太好！

事实上，这个结果很差！

因为仅仅用了一个非常简单的模型。

不过，做一些小小的改进，就能得到0.97的正确率；最好的模型甚至可以超过0.997的正确率！

参考[各种模型的性能对比](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)了解详情。

对于初学者，比结果更重要的是学习到模型的设计思想。若觉得本文有些小儿科，可以查看[高级篇](./TF的MNIST提高)，在那里可以学习到如何用TF构建更加复杂的模型以获得更好的性能。